In [1]:
%pip install -q -U torch==2.0.1 bitsandbytes==0.40.2
%pip install -q -U transformers==4.35.2 peft==0.4.0 accelerate==0.21.0
%pip install -q -U datasets py7zr einops tensorboardX
!pip install evaluate
# Add installed cuda runtime to path for bitsandbytes
import os
import nvidia

cuda_install_dir = '/'.join(nvidia.__file__.split('/')[:-1]) + '/cuda_runtime/lib/'
os.environ['LD_LIBRARY_PATH'] =  cuda_install_dir


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip ins

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import pandas as pd
import os
import torch
import numpy as np
from scipy import stats
# Path to your env.txt file
env_file_path = 'data/env.txt'

# Read and set environment variables
with open(env_file_path, 'r') as file:
    for line in file:
        key, value = line.strip().split('=')
        os.environ[key] = value
token = os.environ['huggingface_token']

## Check Length of MATHWELL Training Data and Prompts

In [7]:
df = pd.read_csv('data/mathwell_annotations_final.csv')
model_path = "meta-llama/Llama-2-70b-hf"   # Specify the path to the model
tokenizer = AutoTokenizer.from_pretrained(model_path, token=token)

In [6]:
question_lengths = 0
for i in range(0, len(df)):
    output = df.iloc[i]['output']
    try: 
        inputs = tokenizer.encode(output, return_tensors="pt")
    except:
        pass
    length = inputs.shape[1]
    question_lengths+=length
print(f"Average question length in tokens: {question_lengths/len(df)}")

Average question length in tokens: 191.32467532467533


In [7]:
df = pd.read_csv('data/mathwell.csv')
import random
topics = ['Superman', "Batman", "Wonder Woman", "Barbie", "Power Rangers", "basketball", "soccer", "football", "volleyball", 'field hockey',\
'Fortnite', 'Spiderman', "Iron Man", "Captain America", "Captain Marvel", "Thor, the God of Thunder", "Ninja Turtles", "Black Panther", "Taylor Swift", "swimming",\
"Pokémon", "Super Mario", "Naruto", "unicorns", "Hello Kitty", "Minecraft", "lacrosse", "cheer leading", "LeBron James", "Steph Curry", "Patrick Mahomes",\
"Serena Williams", "dogs", "cats", "dinosaurs", "Harry Potter", "cars", "planes", "trains", "pizza", "cookies", "ice cream", 'candy']
question_lengths = 0
for i in range(0,1000):
    topic = random.choice(topics)
    final_prompt = f"Write a grade school math word problem about {topic} and Python function with a commented out step-by-step solution to solve the word problem."
    prompt = "Write a grade school math word problem and Python function with a commented out step-by-step solution to solve the word problem."
    questions = []
    for i in range(0, 8):
        question = df['output'].iloc[random.randint(0,len(df)-1)]
        questions.append(question)
    formatted_prompt = []
    for i in range(0,8):
        formatted_prompt.append((f"Below is an instruction that describes a task. "
                f"Write a response that appropriately completes the request.\n\n"
                f"### Instruction:\n{prompt}\n\n### Response: {questions[i]}"))
    formatted_prompt.append(f"Below is an instruction that describes a task. "
                f"Write a response that appropriately completes the request.\n\n"
                f"### Instruction:\n{final_prompt}\n\n### Response: ")
    formatted_prompt = "\n".join(formatted_prompt)
    inputs = tokenizer.encode(formatted_prompt, return_tensors="pt")
    length = inputs.shape[1]
    question_lengths+=length
print(f"Average input length in tokens: {question_lengths/1000}")

Average input length in tokens: 2045.103


In [8]:
def check_length(df):
    question_lengths = []
    for i in range(0, len(df)):
        output = df.iloc[i]['text']
        try: 
            inputs = tokenizer.encode(output, return_tensors="pt")
        except:
            pass
        length = inputs.shape[1]
        question_lengths.append(length)
    return question_lengths
def check_mathwell(df):
    question_lengths = []
    for i in range(0, len(df)):
        output = df.iloc[i]['question']
        try: 
            inputs = tokenizer.encode(output, return_tensors="pt")
        except:
            pass
        length = inputs.shape[1]
        question_lengths.append(length)
    return question_lengths
def check_length_good(df):
    question_lengths = []
    for i in range(0, len(df)):
        output = df.iloc[i]['question']
        try: 
            inputs = tokenizer.encode(output, return_tensors="pt")
        except:
            pass
        length = inputs.shape[1]
        question_lengths.append(length)
    return question_lengths
def check_gsm8k(df):
    question_lengths = []
    for i in range(0, len(df)):
        output = str(df.iloc[i]['instruction'])
        try: 
            inputs = tokenizer.encode(output, return_tensors="pt")
        except:
            pass
        length = inputs.shape[1]
        if length>2:
            question_lengths.append(length)
    return question_lengths

## Check Average Length of GSM8K Training Questions

In [9]:
gsm8k = pd.read_csv('data/gsm8k_questions.csv')
gsm8k = check_gsm8k(gsm8k)
print(f'Average overall question length in tokens: {np.mean(gsm8k)} Standard Deviation: {np.std(gsm8k)}')

Average overall question length in tokens: 65.66239462682475 Standard Deviation: 23.70797156147507


## Check Length of MATHWELL Training Data, Questions Only

In [12]:
mathwell_an = pd.read_csv('data/mathwell_annotations_final.csv')
mathwell_an_len = check_mathwell(mathwell_an)
print(f'Average overall question length in tokens: {np.mean(mathwell_an_len)} Standard Deviation: {np.std(mathwell_an_len)}')

Average overall question length in tokens: 58.95063145809414 Standard Deviation: 16.551501822337883


In [13]:
mathwell_train = mathwell_an[mathwell_an['good']==1]
mathwell_train = check_length_good(mathwell_train)
print(f'Average good question length in tokens: {np.mean(mathwell_train)} Standard Deviation: {np.std(mathwell_train)}')

Average good question length in tokens: 57.259312320916905 Standard Deviation: 15.729965098829148


## Check Length of MATHWELL Final Questions

In [12]:
mathwell_all = pd.read_csv('data/mathwell_solvability.csv')
mathwell = check_length(mathwell_all)
print(f'Average overall question length in tokens: {np.mean(mathwell)} Standard Deviation: {np.std(mathwell)}')

Average overall question length in tokens: 55.24 Standard Deviation: 13.84826342903687


In [13]:
mathwell_good = pd.read_csv('data/mathwell_good.csv')
mathwell_good = check_length_good(mathwell_good)
print(f'Average good question length in tokens: {np.mean(mathwell_good)} Standard Deviation: {np.std(mathwell_good)}')

Average good question length in tokens: 54.537234042553195 Standard Deviation: 13.804628302212464


## Check Length of Llama Questions

In [14]:
llama_all = pd.read_csv('data/llama_solvability.csv')
llama = check_length(llama_all)
print(f'Average overall question length in tokens: {np.mean(llama)} Standard Deviation: {np.std(llama)}')

Average overall question length in tokens: 53.08 Standard Deviation: 15.637442246096388


In [15]:
llama_good = pd.read_csv('data/llama_good.csv')
llama_good = check_length_good(llama_good)
print(f'Average good question length in tokens: {np.mean(llama_good)} Standard Deviation: {np.std(llama_good)}')

Average good question length in tokens: 51.547770700636946 Standard Deviation: 14.40870484412119


## Check Length of LLEMA Questions

In [16]:
llema_all = pd.read_csv('data/llema_solvability.csv')
llema = check_length(llema_all)
print(f'Average overall question length in tokens: {np.mean(llema)} Standard Deviation: {np.std(llema)}')

Average overall question length in tokens: 56.516 Standard Deviation: 22.56549897520549


In [17]:
llema_good = pd.read_csv('data/llema_good.csv')
llema_good = check_length_good(llema_good)
print(f'Average good question length in tokens: {np.mean(llema_good)} Standard Deviation: {np.std(llema_good)}')

Average good question length in tokens: 51.1219512195122 Standard Deviation: 17.264364993586263


## Check Length of MAmmoTH Questions

In [18]:
mammoth_all = pd.read_csv('data/mammoth_solvability.csv')
mammoth = check_length(mammoth_all)
print(f'Average overall question length in tokens: {np.mean(mammoth)} Standard Deviation: {np.std(mammoth)}')

Average overall question length in tokens: 46.056 Standard Deviation: 17.86809626121373


In [19]:
mammoth_good = pd.read_csv('data/mammoth_good.csv')
mammoth_good = check_length_good(mammoth_good)
print(f'Average good question length in tokens: {np.mean(mammoth_good)} Standard Deviation: {np.std(mammoth_good)}')

Average good question length in tokens: 43.957142857142856 Standard Deviation: 13.371008845671309


## Check Length of NumGLUE

In [49]:
numglue = pd.read_csv('data/numglue_questions.csv')
numglue_len = check_gsm8k(numglue)
print(f'Average overall question length in tokens: {np.mean(numglue_len)} Standard Deviation: {np.std(numglue_len)}')

Average overall question length in tokens: 145.77979749661165 Standard Deviation: 136.64416090041578


## Check Length of ASDIV

In [24]:
asdiv = pd.read_csv('data/ASDiv_clean.csv')
asdiv_len = check_length_good(asdiv)
print(f'Average overall question length in tokens: {np.mean(asdiv_len)} Standard Deviation: {np.std(asdiv_len)}')

Average overall question length in tokens: 45.06898047722343 Standard Deviation: 15.753717766689356


## Check Length of SVAMP

In [26]:
svamp = pd.read_json('data/SVAMP.json')
svamp['question'] = svamp['Body'] + " " + svamp['Question']
svamp_len = check_length_good(svamp)
print(f'Average overall question length in tokens: {np.mean(svamp_len)} Standard Deviation: {np.std(svamp_len)}')

Average overall question length in tokens: 47.268 Standard Deviation: 11.699067313251941


## Check Length of GSM-Hard

In [27]:
gsm_hard = pd.read_json('data/gsmhard.json')
gsm_hard_len = check_length_good(gsm_hard)
print(f'Average overall question length in tokens: {np.mean(gsm_hard_len)} Standard Deviation: {np.std(gsm_hard_len)}')

Average overall question length in tokens: 72.9294920394238 Standard Deviation: 25.636173190635287


## Tests for Statistical Significance

In [28]:
# Perform an independent two-sample t-test on solvability
t_statistic, p_value = stats.ttest_ind(llema, mathwell)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

T-statistic: 0.7604991428423419
P-value: 0.44731618802821826


In [29]:
# Perform an independent two-sample t-test on solvability
t_statistic, p_value = stats.ttest_ind(gsm8k, mathwell_train)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

T-statistic: 14.558138817920046
P-value: 1.0248750555174296e-47


In [30]:
# Perform an independent two-sample t-test on solvability
t_statistic, p_value = stats.ttest_ind(llema, mathwell)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

T-statistic: 0.7604991428423419
P-value: 0.44731618802821826


In [31]:
# Perform an independent two-sample t-test on solvability
t_statistic, p_value = stats.ttest_ind(llama_good, mathwell_good)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

T-statistic: -1.957775025575982
P-value: 0.05106687191095604
